In [ ]:
import os
from fair import FAIR
from fair.io import read_properties
from fair.interface import initialise
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl

In [ ]:
f = FAIR()

In [ ]:
f.define_time(1750, 2501, 1)
f.define_scenarios(["high-extension", "high-overshoot", "medium-extension", "medium-overshoot", "low", "verylow", "verylow-overshoot"])
species, properties = read_properties('../data/fair-parameters/species_configs_properties_1.4.1.csv')
f.define_species(species, properties)
f.ch4_method='Thornhill2021'
df_configs = pd.read_csv('../data/fair-parameters/calibrated_constrained_parameters_1.4.1.csv', index_col=0)
f.define_configs(df_configs.index)
f.allocate()

In [ ]:
f.fill_from_csv(
    emissions_file='../data/emissions/extensions_1750-2500.csv',
    forcing_file='../data/forcing/volcanic_solar.csv',
)

In [ ]:
colors = {
    'high-extension': '#800000',
    'high-overshoot': '#ff0000',
    'medium-extension': '#c87820',
    'medium-overshoot': '#d3a640',
    'low': '#098740',
    'verylow': '#0080d0',
    'verylow-overshoot': '#100060',
}

In [ ]:
os.makedirs('../plots', exist_ok=True)

In [ ]:
fig, ax = pl.subplots()
for scenario in f.scenarios:
    ax.plot(
        f.timepoints,
        (
            f.emissions.sel(scenario=scenario, specie="CO2 FFI", config=f.configs[0]) +
            f.emissions.sel(scenario=scenario, specie="CO2 AFOLU", config=f.configs[0])
        ),
        label=scenario,
        color=colors[scenario]
    )
ax.set_ylabel("CO$_2$ emissions, GtCO$_2$ yr$^{-1}$")
ax.axhline(ls=':', color='k', lw=0.5)
ax.legend();
pl.savefig('../plots/co2_emissions.png')

In [ ]:
df_emis = pd.read_csv('../data/emissions/extensions_1750-2500.csv')#
df_emis.loc[df_emis['variable']=="GHG", "1750.5"]

In [ ]:
fig, ax = pl.subplots()
for scenario in f.scenarios:
    ax.plot(
        f.timepoints,
        df_emis.loc[(df_emis['variable']=="GHG") & (df_emis['scenario']==scenario), "1750.5":].squeeze() / 1e6,
        label=scenario,
        color=colors[scenario]
    )
ax.set_ylabel("GHG emissions, GtCO$_2$eq yr$^{-1}$")
ax.axhline(ls=':', color='k', lw=0.5)
ax.legend();
pl.savefig('../plots/ghg_emissions.png')

In [ ]:
f.fill_species_configs('../data/fair-parameters/species_configs_properties_1.4.1.csv')

In [ ]:
f.override_defaults('../data/fair-parameters/calibrated_constrained_parameters_1.4.1.csv')

In [ ]:
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)
initialise(f.ocean_heat_content_change, 0)

In [ ]:
f.run()

In [ ]:
fig, ax = pl.subplots()
for scenario in f.scenarios:
    ax.fill_between(
        f.timebounds,
        (
            f.temperature.sel(scenario=scenario, layer=0)-
            f.temperature.sel(scenario=scenario, layer=0, timebounds=np.arange(1850,1902)).mean(dim='timebounds')
        ).quantile(0.05, dim='config'),
        (
            f.temperature.sel(scenario=scenario, layer=0)-
            f.temperature.sel(scenario=scenario, layer=0, timebounds=np.arange(1850,1902)).mean(dim='timebounds')
        ).quantile(0.95, dim='config'),
        color=colors[scenario],
        lw=0,
        alpha=0.1
    )
    ax.plot(
        f.timebounds,
        (
            f.temperature.sel(scenario=scenario, layer=0)-
            f.temperature.sel(scenario=scenario, layer=0, timebounds=np.arange(1850,1902)).mean(dim='timebounds')
        ).median(dim='config'),
        label=scenario,
        color=colors[scenario]
    )
ax.axhline(0, ls=':', color='k', lw=0.5)
ax.set_ylabel('temperature above 1850-1900, K')
pl.legend();
pl.savefig('../plots/temperature.png')

In [ ]:
fig, ax = pl.subplots()
for scenario in f.scenarios:
    ax.plot(
        f.timebounds,
        f.concentration.sel(scenario=scenario, specie="CO2").median(dim='config'),
        label=scenario,
        color=colors[scenario]
    )
ax.set_ylabel('CO$_2$ concentration, ppm')
pl.legend();
pl.savefig('../plots/co2_concentration.png')

In [ ]:
fig, ax = pl.subplots()
for scenario in f.scenarios:
    pl.plot(
        f.timebounds,
        f.forcing_sum.sel(scenario=scenario).median(dim='config'),
        label=scenario,
        color=colors[scenario]
    )
ax.set_ylabel('Effective radiative forcing, W m$^{-2}$')
pl.legend();
pl.savefig('../plots/forcing.png')